In [13]:
from agno.agent import Agent

from agno.models.openai import OpenAIChat
from agno.models.google import Gemini
from agno.team import Team
from agno.tools.firecrawl import FirecrawlTools
import datetime
from agno.tools.tavily import TavilyTools
from dotenv import load_dotenv
from agno.models.groq import Groq
load_dotenv(override=True)

# Model = OpenAIChat("gpt-4o-mini")
# Model = Groq("meta-llama/llama-4-scout-17b-16e-instruct")
market_research_model = OpenAIChat("gpt-4o-mini")
market_analysis_model = OpenAIChat("gpt-4o-mini")
report_generator_model = OpenAIChat("gpt-4.1-mini-2025-04-14")
manager_model = OpenAIChat("gpt-4o-mini")
mermaid_model = Gemini("gemini-2.5-flash-preview-04-17")

# Market Researcher agent that uses Tavily to search and Firecrawl to scrape data
market_researcher = Agent(
    name="MarketResearcher",
    model=market_research_model,
    role="Searches and scrapes data on existing platforms, features, and pricing",
    instructions = [
        "Given a project description, generate 3–5 relevant search terms to identify key competitors in the target market.",
        "Use Tavily to search for each term and compile a list of the top competitors.",
        "Use Firecrawl to map URLs from the identified competitor websites.",
        "Scrape each URL using Firecrawl, focusing on extracting detailed information about features, pricing plans, and unique selling points (USPs).",
        "Ensure that comprehensive data is collected for at least 5 major competitors.",
        "Emphasize extracting clear data points related to product/service features, pricing structure, and market positioning.",
        "Organize the extracted data in a clean, structured format suitable for comparison and analysis."
    ],

    #add crawl true if needed
    tools=[TavilyTools(), FirecrawlTools(scrape=True, mapping=True)],
    add_datetime_to_instructions=True,
    show_tool_calls=True,
    markdown=True
)
# Market Analyzer agent that uses reasoning to analyze the collected data
market_analyzer = Agent(
    name="MarketAnalyzer",
    model=market_analysis_model,
    role="Analyzes market data and identifies trends, gaps, and opportunities",
    description=(
        "You are a market analysis expert. Given data on competitors, features, and pricing, "
        "your goal is to analyze the market landscape and identify opportunities for the project."
    ),
    instructions=[
        "Analyze the data collected by the Market Researcher to identify market trends and patterns.",
        "Compare features across different platforms to identify common offerings and unique differentiators.",
        "Analyze pricing strategies to understand market positioning and value propositions.",
        "Identify gaps in the market that could be potential opportunities for the project.",
        "Evaluate the competitive landscape to determine market saturation and entry barriers.",
        "Provide data-driven insights on market positioning and potential unique selling points.",
    ],
    reasoning=True,  # Enable Agno's reasoning capabilities
    add_datetime_to_instructions=True,
    show_tool_calls=True
)

# Report Generator agent that creates a comprehensive market validation report
report_generator = Agent(
    name="ReportGenerator",
    model=report_generator_model,
    role="Creates a comprehensive market validation report in Markdown format",
    description=(
        "You are a professional report writer. Given analyzed market data, "
        "your goal is to create a comprehensive, well-structured market validation report."
    ),
    instructions = [
        "Create a professional market validation report in Markdown format.",
        "Include an executive summary highlighting key findings and recommendations.",
        "Provide detailed competitor analysis with feature comparisons and pricing plans.",
        "Include a comparison table of at least 3–5 competitors, highlighting their strengths and weaknesses.",
        """Visualize the competitor comparison using a Mermaid chart:
        ```mermaid
        pie
            title Competitor Mindshare
            "Elastic Search" : 15.6
            "Amazon Kendra" : 15.8
            "Azure Search" : 14.3
        ```""",
        "Identify and recommend unique selling points (USPs) of the project that are not offered by competitors.",
        """Summarize USPs using a Mermaid chart:
        ```mermaid
        graph LR
            A[Niche Industry Customization] --> B[Increased Adoption]
            C[Enhanced User Analytics] --> D[Improved User Experience]
            E[AI-Driven Personalization] --> F[Enhanced User Engagement]
            G[Flexible Pricing Models] --> H[Broader Market Reach]
        ```""",
        "Provide a market projection for the next 3–5 years, including estimated market size and growth trends.",
        """Visualize market projection using a Mermaid chart:
        ```mermaid
        xychart-beta
            title "Market Size Projection"
            x-axis "Year" [2025, 2026, 2027, 2028, 2029, 2030]
            y-axis "Market Size (Billion USD)" 0 --> 5.5
            line [2.5, 2.875, 3.3, 3.8, 4.4, 5.0]
        ```""",
        "Define potential revenue streams for the project and estimate earning potential over 1–3 years.",
        """Summarize revenue stream projections using a Mermaid pie chart, example:
        ```mermaid
            pie
            title Revenue Streams
            "Subscriptions": 40
            "Marketplace Fees": 25
            "Ads": 20
            "Enterprise Licensing": 15
        ```""",
        "Estimate initial costs for building the project, including human resources, tools, and time required.",
        """Visualize estimated cost breakdown using a Mermaid flowchart, example:
        ```mermaid
        flowchart TD
        A[Project Cost Estimate] --> B[Human Resources]
        A --> C[Tools & Software]
        A --> D[Time Allocation]
        B --> B1[Developers - $20k]
        B --> B2[Designers - $10k]
        C --> C1[Cloud Services - $5k]
        C --> C2[APIs/Tools - $3k]
        D --> D1[3 Months Timeline]
        ```""",
        "Format the report with clear headings, subheadings, tables, and bullet points for readability.",
        "Ensure the report is comprehensive yet concise, focusing on actionable insights and business value."
    ],

    add_datetime_to_instructions=True,
    show_tool_calls=True,
    markdown=True
)


# Market Validation Team that coordinates the entire process
market_validation_team = Team(
    name="MarketValidationTeam",
    mode="coordinate",
    model=manager_model,  # Using GPT-4o-mini for coordination
    members=[market_researcher, market_analyzer, report_generator, mermaid_expert],
    description="You are a market validation team. Given a project description, your goal is to produce a comprehensive market validation report.",
    instructions=[
        "First, ask the Market Researcher to search for and collect data on existing platforms, features, and pricing.",
        "The researcher should first use Tavily to find relevant competitor websites, then use Firecrawl to scrape detailed information.",
        "Then, ask the Market Analyzer to analyze the data and identify trends, gaps, and opportunities.",
        "ask the Report Generator to create a comprehensive market validation report.",
        """Ensure the final report includes : 
        1. competitor analysis with feature and pricing plans comparision.
        2. recommended USPs that not offered by competitors. 
        3. Future Market Projection.
        4. Revenue Stream and the Potention of the Revenue.
        5. Initial Cost Estimate to  produce the MVP(Minimum Viable Product).""",
        "Expected output: A comprehensive market validation report, avoid formal introductions, small talk, or unnecessary closing statements. Start directly with the content of the report. Do not use phrases like 'This report provides...' or 'feel free to ask'."
    ],
    add_datetime_to_instructions=True,
    add_member_tools_to_system_message=True,
    enable_agentic_context=True,  # Allow the team to maintain a shared context
    share_member_interactions=True,  # Share all member responses with subsequent member requests
    show_members_responses=True,
    show_tool_calls=True,
    markdown=True,
)

In [14]:
project_description = """
    Notion AI Agent, AI agent that can answer questions, create, update, and delete pages in Notion.
    """
markdown_report = market_validation_team.run(project_description)

In [4]:
from agno.run.team import TeamRunResponse

In [15]:
print(markdown_report.content)

# Market Validation Report  
**Notion AI Agent Platforms**  
*Date: May 2025*

---

## Executive Summary

This market validation report analyzes existing Notion AI Agent platforms to identify current market trends, gaps, opportunities, and strategic recommendations. Our research reveals an active competitive space primarily focused on automating workflows and enhancing productivity within Notion. However, evident gaps exist in offering niche industry customization, AI-driven actionable insights, and transparent pricing models.

We recommend developing a Notion AI Agent that uniquely combines:

- Industry-specific customization tailored to niche markets.
- Advanced AI-driven data insights and predictive analytics.
- Transparent, flexible pricing tiers to maximize reach.

These USPs will differentiate from competitors and align with growing user demands. Market projections anticipate steady growth over the next five years, opening revenue opportunities via subscriptions, premium features

In [6]:
len(markdown_report.messages)

11

In [7]:
for i, j in enumerate(markdown_report.messages):
    print(f"""\n\n
----
no: {i}
type: {j.role}
input_token: {j.metrics.input_tokens}
output_token: {j.metrics.output_tokens}
total_token: {j.metrics.total_tokens}
tool_call: {j.tool_calls}
----""")
    print(j.content)




----
no: 0
type: system
input_token: 0
output_token: 0
total_token: 0
tool_call: None
----
You are the leader of a team and sub-teams of AI Agents.
Your task is to coordinate the team to complete the user's request.

Here are the members in your team:
<team_members>
 - Agent 1:
   - ID: market-researcher
   - Name: MarketResearcher
   - Role: Searches and scrapes data on existing platforms, features, and pricing
   - Member tools:
    - web_search_using_tavily
    - scrape_website
    - map_website
 - Agent 2:
   - ID: market-analyzer
   - Name: MarketAnalyzer
   - Role: Analyzes market data and identifies trends, gaps, and opportunities
 - Agent 3:
   - ID: report-generator
   - Name: ReportGenerator
   - Role: Creates a comprehensive market validation report in Markdown format
 - Agent 4:
   - ID: mermaid-expert
   - Name: MermaidExpert
   - Role: Expert in mermaid charts and diagrams
</team_members>

<how_to_respond>
- You can either respond directly or transfer tasks to members 

In [6]:
markdown_report.member_responses[0].messages

[Message(role='system', content='<your_role>\nSearches and scrapes data on existing platforms, features, and pricing\n</your_role>\n\n<instructions>\n- Given a project description, first generate 3-5 search terms to find key competitors in this market.\n- Use Tavily to search for each term and identify the top competitors in the market space.\n- Then use Firecrawl to map url from competitor websites\n- After that use Firecrawl to scrape data from the url we get from the mapping process focusing on features, pricing, and unique selling points.Collect comprehensive data on at least 5 major competitors in the market space.\n- Focus on extracting specific information about features, pricing plans, and market positioning.\n- Organize the collected data in a structured format for analysis.\n</instructions>\n\n<additional_information>\n- Use markdown to format your answers.\n- The current time is 2025-05-14 01:33:42.094525.\n</additional_information>', name=None, tool_call_id=None, tool_calls

In [27]:
for i, j in enumerate(markdown_report.member_responses[0].messages):
    print(f"""\n\n
----
no: {i}
type: {j.role}
input_token: {j.metrics.input_tokens}
output_token: {j.metrics.output_tokens}
total_token: {j.metrics.total_tokens}
tool_call: {j.tool_calls}
----""")
    print(j.content)




----
no: 0
type: system
input_token: 0
output_token: 0
total_token: 0
tool_call: None
----
<your_role>
Searches and scrapes data on existing platforms, features, and pricing
</your_role>

<instructions>
- Given a project description, generate 3–5 relevant search terms to identify key competitors in the target market.
- Use Tavily to search for each term and compile a list of the top competitors.
- Use Firecrawl to map URLs from the identified competitor websites.
- Scrape each URL using Firecrawl, focusing on extracting detailed information about features, pricing plans, and unique selling points (USPs).
- Ensure that comprehensive data is collected for at least 5 major competitors.
- Emphasize extracting clear data points related to product/service features, pricing structure, and market positioning.
- Organize the extracted data in a clean, structured format suitable for comparison and analysis (e.g., table or JSON).
</instructions>

<additional_information>
- Use markdown to form

In [28]:
for i, j in enumerate(markdown_report.member_responses[1].messages):
    print(f"""\n\n
----
no: {i}
type: {j.role}
input_token: {j.metrics.input_tokens}
output_token: {j.metrics.output_tokens}
total_token: {j.metrics.total_tokens}
tool_call: {j.tool_calls}
----""")
    print(j.content)




----
no: 0
type: system
input_token: 0
output_token: 0
total_token: 0
tool_call: None
----
You are a market analysis expert. Given data on competitors, features, and pricing, your goal is to analyze the market landscape and identify opportunities for the project.

<your_role>
Analyzes market data and identifies trends, gaps, and opportunities
</your_role>

<instructions>
- Analyze the data collected by the Market Researcher to identify market trends and patterns.
- Compare features across different platforms to identify common offerings and unique differentiators.
- Analyze pricing strategies to understand market positioning and value propositions.
- Identify gaps in the market that could be potential opportunities for the project.
- Evaluate the competitive landscape to determine market saturation and entry barriers.
- Provide data-driven insights on market positioning and potential unique selling points.
</instructions>

<additional_information>
- Use markdown to format your answe

In [29]:
for i, j in enumerate(markdown_report.member_responses[2].messages):
    print(f"""\n\n
----
no: {i}
type: {j.role}
input_token: {j.metrics.input_tokens}
output_token: {j.metrics.output_tokens}
total_token: {j.metrics.total_tokens}
tool_call: {j.tool_calls}
----""")
    print(j.content)




----
no: 0
type: system
input_token: 0
output_token: 0
total_token: 0
tool_call: None
----
You are a professional report writer. Given analyzed market data, your goal is to create a comprehensive, well-structured market validation report.

<your_role>
Creates a comprehensive market validation report in Markdown format
</your_role>

<instructions>
- Create a professional market validation report in Markdown format.
- Include an executive summary highlighting key findings and recommendations.
- Provide detailed competitor analysis with feature comparisons and pricing plans.
- Include a comparison table of at least 3–5 competitors, highlighting their strengths and weaknesses.
- Visualize the competitor comparison using a Mermaid bar chart:

```mermaid
bar
title Competitor Feature Coverage
x-axis Feature Coverage %
"Your Project": 95
"Competitor A": 80
"Competitor B": 70
"Competitor C": 60
```
- Identify and recommend unique selling points (USPs) of the project that are not offered by c

In [36]:
for i, j in enumerate(markdown_report.member_responses[3].messages):
    print(f"""\n\n
----
no: {i}
type: {j.role}
input_token: {j.metrics.input_tokens}
output_token: {j.metrics.output_tokens}
total_token: {j.metrics.total_tokens}
tool_call: {j.tool_calls}
----""")
    print(j.content)




----
no: 0
type: system
input_token: 0
output_token: 0
total_token: 0
tool_call: None
----
<your_role>
Expert in mermaid charts and diagrams
</your_role>

<instructions>
- Scan the given Markdown document and extract all Mermaid code blocks.
- Validate the syntax of each Mermaid code block.
- If any syntax errors are found, correct them to ensure the diagram renders properly.
- Ensure all Mermaid charts and diagrams match the intended chart type (e.g., bar, pie, flowchart).
- Check for required structure elements such as titles, axis labels, and proper indentation where needed.
- Replace invalid Mermaid blocks with corrected versions, preserving the original intent and data.
- Return the Markdown document with all Mermaid diagrams fixed and valid.
</instructions>

<additional_information>
- Use markdown to format your answers.
- The current time is 2025-05-14 12:09:03.531673.
</additional_information>



----
no: 1
type: user
input_token: 0
output_token: 0
total_token: 0
tool_call: 

In [61]:
from agno.run.response import RunResponse

In [41]:
print(markdown_report.content)

Here is the comprehensive market validation report for your "Search as a Service" project focused on marketplace product search:

## Market Validation Report: Search as a Service for Marketplace Product Search

**Date:** 2025-05-13

**Prepared for:** [Project Stakeholders]

**Prepared by:** [Your Name/Team]

---

### 1. Executive Summary

This report presents the findings of a market validation analysis for a proposed 'Search as a Service' platform specifically designed for marketplace product search. The analysis evaluated the competitive landscape, key market trends, pricing strategies, and identified significant gaps and opportunities.

The market for e-commerce Search as a Service is competitive, dominated by players like Algolia, Coveo, Elastic, Bloomreach, and Constructor. A strong trend towards AI/ML-driven search, personalization, and the adoption of Generative AI is evident. Competitors offer comprehensive feature suites, including merchandising tools, analytics, and personali

In [16]:
def save_markdown(markdown_content, base_filename):
    """
    Save markdown content to a file with a timestamp in the filename.
    
    Args:
        markdown_content (str): The markdown content to save
        base_filename (str): The base name of the file without extension
    """
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{base_filename}_{timestamp}.md"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    
    return filename

In [17]:
save_markdown(markdown_report.content, "./result/market_validation_report")

'./result/market_validation_report_20250514_225557.md'

In [ ]:
from agno.agent import Agent
from agno.models.google import Gemini

mermaid_model = Gemini("gemini-2.5-flash-preview-04-17")

mermaid_expert = Agent(
    name="MermaidExpert",
    model=report_generator_model,
    role="Expert in mermaid charts and diagrams",
    instructions = [
        "Scan the given Markdown document and extract all Mermaid code blocks.",
        "Validate the syntax of each Mermaid code block.",
        "If any syntax errors are found, correct them to ensure the diagram renders properly.",
        "Ensure all Mermaid charts and diagrams match the intended chart type (e.g., bar, pie, flowchart).",
        "Check for required structure elements such as titles, axis labels, and proper indentation where needed.",
        "Replace invalid Mermaid blocks with corrected versions, preserving the original intent and data.",
        "Return the Markdown document with all Mermaid diagrams fixed and valid."
    ],

    add_datetime_to_instructions=True,
    show_tool_calls=True
)


mermaid_response = mermaid_expert.run("""# Market Validation Report: Search as a Service Platforms


Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x7f70ff3082c0>
Traceback (most recent call last):
  File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/openai/_base_client.py", line 801, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/httpx/_client.py", line 228, in is_closed
    return self._state == ClientState.CLOSED
           ^^^^^^^^^^^
AttributeError: 'SyncHttpxClientWrapper' object has no attribute '_state'
Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x7f70ff3082c0>
Traceback (most recent call last):
  File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/openai/_base_client.py", line 801, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/httpx/_client.py", line 228, in is_closed
    return self._state == ClientState.CLOSED
           ^^^^^^^^^^^
AttributeError: 'SyncHttpxClientWr

In [40]:
mermaid_response.content

'```markdown\n# Market Validation Report: Search as a Service Platforms\n\n---\n\n## Executive Summary\n\nThis report provides a comprehensive validation of the Search as a Service (SaaS) market based on collected data and thorough analysis of leading platforms in the industry. Currently, the market is dominated by established platforms like Amazon CloudSearch and Elastic Search, which excel in scalability and advanced features such as machine learning and multi-source integration.\n\n**Key Findings:**\n\n- High demand for scalable, real-time, and accurate search functionalities.\n- Machine learning integration is a core trend to deliver contextualized results.\n- Most platforms lack intuitive, user-friendly interfaces, especially for non-technical users.\n- Niche industry-specific customization remains underexplored.\n- Pricing models vary, with a trend toward fully managed cloud services and tiered subscription plans.\n\n**Recommendations:**\n\n- Focus on developing a seamless, intui

In [42]:
save_markdown(mermaid_response.content, "./result/mermaid_report")

'./result/mermaid_report_20250514_123548.md'

In [57]:
from agno.agent import Agent

from agno.models.openai import OpenAIChat
from agno.models.google import Gemini
from agno.team import Team
from agno.tools.firecrawl import FirecrawlTools
import datetime
from agno.tools.tavily import TavilyTools
from dotenv import load_dotenv
from agno.models.groq import Groq
load_dotenv(override=True)

# Model = OpenAIChat("gpt-4o-mini")
# Model = Groq("meta-llama/llama-4-scout-17b-16e-instruct")
market_research_model = OpenAIChat("gpt-4o-mini")
market_analysis_model = OpenAIChat("gpt-4o-mini")
report_generator_model = OpenAIChat("gpt-4.1-mini-2025-04-14")
manager_model = OpenAIChat("gpt-4o-mini")
mermaid_model = Gemini("gemini-2.5-flash-preview-04-17")


competitor_analysis = Agent(
    name="CompetitorAnalyst",
    model=market_analysis_model,
    role="Analyzes competitor data and identifies trends, gaps, and opportunities",
    instructions = [
        "Use Firecrawl to map URLs from the identified competitor websites.",
        "Scrape each URL using Firecrawl, focusing on extracting detailed information about features, pricing plan(included their price), and unique selling points (USPs).",
        "Ensure you collected data such as features, price for every Tier(pricing plan), and unique selling points (USPs).",
        "Organize the extracted data in a clean, structured format suitable for comparison and analysis."
    ],
    tools=[FirecrawlTools(scrape=True, mapping=True)],
    reasoning=True,
    add_datetime_to_instructions=True,
    show_tool_calls=True,
    markdown=True,
)

# Market Researcher agent that uses Tavily to search and Firecrawl to scrape data
market_researcher = Team(
    name="MarketResearcherTeam",
    model=market_research_model,
    members=[competitor_analysis],
    instructions = [
        "Given a project description, generate 3–5 relevant search terms to identify key competitors in the target market.",
        "Use Tavily to search for each term and compile a list of the top competitors.",
        "Choose 5 major competitors from the tavily search result.",
        "Give the competitor analyst ONLY ONE of the competitors to analyze.",
        "Every time competitor analyst Done the analysis, ask the competitor analyst to analyze the next competitor, do it until all competitors are analyzed.",
    ],

    #add crawl true if needed
    tools=[TavilyTools()],
    add_datetime_to_instructions=True,
    add_member_tools_to_system_message=True,
    enable_agentic_context=True,  # Allow the team to maintain a shared context
    share_member_interactions=True,  # Share all member responses with subsequent member requests
    show_members_responses=True,
    show_tool_calls=True,
    markdown=True,
    reasoning=True,
)

In [ ]:
project_description = """
    Search as a Service project, service that provide search results like marketplace product search.
    """
result = market_researcher.run(project_description)

WARNING  Could not run function scrape_website(url=...)

ERROR    Internal Server Error: Failed to scrape URL. (Internal server error) - All scraping engines failed! --    
         Double check the URL to make sure it's not broken. If the issue persists, contact us at                   
         help@firecrawl.com. - No additional error details provided.                                               
         Traceback (most recent call last):                                                                        
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/agno/tools/function.py", line 560, in     
         execute                                                                                                   
             result = self.function.entrypoint(**arguments)                                                        
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                        
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/agno/tools/firecrawl.py", line 91, in     
         scrape_website                                                                                            
             scrape_result = self.app.scrape_url(url, **params)                                                    
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                    
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/firecrawl/firecrawl.py", line 575, in     
         scrape_url                                                                                                
             self._handle_error(response, 'scrape URL')                                                            
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/firecrawl/firecrawl.py", line 2213, in    
         _handle_error                                                                                             
             raise requests.exceptions.HTTPError(message, response=response)                                       
         requests.exceptions.HTTPError: Internal Server 

WARNING  Could not run function scrape_website(url=...)

ERROR    Internal Server Error: Failed to scrape URL. (Internal server error) - All scraping engines failed! --    
         Double check the URL to make sure it's not broken. If the issue persists, contact us at                   
         help@firecrawl.com. - No additional error details provided.                                               
         Traceback (most recent call last):                                                                        
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/agno/tools/function.py", line 560, in     
         execute                                                                                                   
             result = self.function.entrypoint(**arguments)                                                        
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                        
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/agno/tools/firecrawl.py", line 91, in     
         scrape_website                                                                                            
             scrape_result = self.app.scrape_url(url, **params)                                                    
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                    
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/firecrawl/firecrawl.py", line 575, in     
         scrape_url                                                                                                
             self._handle_error(response, 'scrape URL')                                                            
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/firecrawl/firecrawl.py", line 2213, in    
         _handle_error                                                                                             
             raise requests.exceptions.HTTPError(message, response=response)                                       
         requests.exceptions.HTTPError: Internal Server 

WARNING  Could not run function scrape_website(url=...)

...

WARNING  Could not run function scrape_website(url=...)

ERROR    Unexpected error during scrape URL: Status code 429. Rate limit exceeded. Consumed (req/min): 19,         
         Remaining (req/min): 0. Upgrade your plan at https://firecrawl.dev/pricing for increased rate limits or   
         please retry after 3s, resets at Wed May 14 2025 14:37:24 GMT+0000 (Coordinated Universal Time) - No      
         additional error details provided.                                                                        
         Traceback (most recent call last):                                                                        
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/agno/tools/function.py", line 560, in     
         execute                                                                                                   
             result = self.function.entrypoint(**arguments)                                                        
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                        
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/agno/tools/firecrawl.py", line 91, in     
         scrape_website                                                                                            
             scrape_result = self.app.scrape_url(url, **params)                                                    
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                    
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/firecrawl/firecrawl.py", line 575, in     
         scrape_url                                                                                                
             self._handle_error(response, 'scrape URL')                                                            
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/firecrawl/firecrawl.py", line 2213, in    
         _handle_error                                                                                             
             raise requests.exceptions.HTTPError(message

WARNING  Reasoning error. Reasoning response is empty, continuing regular session...

WARNING  Could not run function scrape_website(url=...)

ERROR    Internal Server Error: Failed to scrape URL. (Internal server error) - All scraping engines failed! --    
         Double check the URL to make sure it's not broken. If the issue persists, contact us at                   
         help@firecrawl.com. - No additional error details provided.                                               
         Traceback (most recent call last):                                                                        
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/agno/tools/function.py", line 560, in     
         execute                                                                                                   
             result = self.function.entrypoint(**arguments)                                                        
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                        
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/pydantic/_internal/_validate_call.py",    
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/agno/tools/firecrawl.py", line 91, in     
         scrape_website                                                                                            
             scrape_result = self.app.scrape_url(url, **params)                                                    
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                    
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/firecrawl/firecrawl.py", line 575, in     
         scrape_url                                                                                                
             self._handle_error(response, 'scrape URL')                                                            
           File "/home/saika/taskflow/.venv/lib/python3.12/site-packages/firecrawl/firecrawl.py", line 2213, in    
         _handle_error                                                                                             
             raise requests.exceptions.HTTPError(message, response=response)                                       
         requests.exceptions.HTTPError: Internal Server 

In [53]:
print(result.content)

### Key Competitors in the "Search as a Service" Market

Here are some prominent players in the "Search as a Service" sector, specifically for marketplace product searches, along with detailed comparisons on features, pricing, and unique selling points:

#### 1. **Elastic Search**
   - **Overview**: A powerful open-source search engine based on Lucene.
   - **Features**:
     - High scalability and performance.
     - Advanced features for full-text search and real-time analytics.
     - Supports various data types (structured and unstructured).
   - **Pricing**: 
     - Basic usage is free; commercial support varies based on the subscription model.
   - **Unique Selling Points (USPs)**:
     - Large community and extensive plugins and integrations.
     - Enterprise-level features with self-hosted and cloud options.

#### 2. **Amazon Kendra**
   - **Overview**: A cloud-based service leveraging machine learning for more accurate search results.
   - **Features**: 
     - Natural Langua